In [2]:
!gcloud dataproc clusters list --region us-central1


NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-862a  GCE       4                                       RUNNING  us-central1-a


In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

Package openjdk-8-jdk-headless is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-8-jdk-headless' has no installation candidate


In [5]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import math
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 11:01 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '316399401' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [9]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text","title","id").rdd


In [44]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

# NUM_BUCKETS = 124
# def token2bucket_id(token):
#   return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  words = [token for token in tokens if token not in all_stopwords]
  counter = Counter()
  counter.update(words)
  final = [(token,(id,counter[token])) for token in counter.keys()]
  # raise NotImplementedError()
  return final

def calc_tfidf(record):
    print("a")
    if (len(record)==0):
        return
    N=len(DL)
    sumArr = 0
    print(record)
    docid=record[0][1][0]
    for rec in record:
        print(rec)
        term = rec[0]
        print(term)
        tf = rec[1][1]/DL[docid]
        print(tf)
        if term not in w2df_dict.keys():
            continue
        df = w2df_dict[term]
        tfidf = tf*math.log(N/df,10)
        tfidfSquared = tfidf**2
        sumArr = sumArr + tfidfSquared
    return (docid,math.sqrt(sumArr))
  

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl,key=lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  # raise NotImplementedError()
  return postings.map(lambda x: (x[0],len(x[1])))



In [17]:
# def max_freq(text):
#   tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#   words = [token for token in tokens if token not in all_stopwords]
#   counter = Counter()
#   counter.update(words)
#   return counter.most_common()[0]   


In [11]:
def tokenize(text):
  #TODO probablly need some fixes
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

In [12]:
doc_wordlist_pairs = doc_text_pairs.map(lambda x: (x[2],len(tokenize(x[0]))+len(tokenize(x[1]))))

DL = doc_wordlist_pairs.collectAsMap()

In [26]:
print(len(DL))

6348910


In [ ]:
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[2]))
# docid_maxfreq_pairs = doc_text_pairs.map(lambda x: (x[2],max_freq(x[0])))
# docid_maxfreq_dict = docid_maxfreq_pairs.collectAsMap()
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
doc_word_counts = doc_text_pairs.map(lambda x: word_count(x[0], x[2]))


In [29]:
print(doc_word_counts)

PythonRDD[32] at RDD at PythonRDD.scala:53


In [ ]:
docid_veclen_pairs = doc_word_counts.filter(lambda x: len(x)!=0)
docid_veclen_pairs = docid_veclen_pairs.map(lambda x: calc_tfidf(x))
docid_veclen_dict = docid_veclen_pairs.collectAsMap()
print(len(docid_veclen_dict))
# partition posting lists and write out


22/01/05 16:39:40 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_32_python !


6347138


In [47]:
#euclidian distance
pv_clean = 'tfidf_vec_len.pkl'
with open(pv_clean, 'wb') as f:
    pickle.dump(docid_veclen_dict, f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"tfidf_vec_len/{pv_clean}")

blob.upload_from_filename(pv_clean)

617.8538153793296
